# TrOCR Fine-tuning for Handwritten Prescription Recognition

This notebook demonstrates how to fine-tune Microsoft's TrOCR (Transformer-based Optical Character Recognition) model specifically for reading handwritten medical prescriptions.

## Overview:
- **Goal**: Extract drug names and dosage information from handwritten prescription images
- **Model**: Microsoft TrOCR (pre-trained on handwritten text)
- **Dataset**: ~100 handwritten prescription images with annotations
- **Output**: Structured text that can be fed to the Zaura Health drug interaction model

## Pipeline:
1. **Data Preparation**: Organize prescription images and create annotations
2. **Model Setup**: Load pre-trained TrOCR model
3. **Fine-tuning**: Train on prescription-specific vocabulary and layout
4. **Evaluation**: Test accuracy on prescription text extraction
5. **Integration**: Create API for Zaura Health website

## Requirements:
- Prescription images in common formats (JPG, PNG)
- Ground truth transcriptions for training
- GPU recommended for training (CPU supported but slower)

In [ ]:
# Install required packages
!pip install transformers[torch] datasets pillow torch torchvision accelerate evaluate nltk rouge-score
!pip install opencv-python matplotlib seaborn tqdm
!pip install --upgrade huggingface_hub

In [ ]:
# Import required libraries
import os
import json
import torch
import numpy as np
import pandas as pd
from PIL import Image, ImageDraw, ImageFont
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import re
from typing import List, Dict, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

# Transformers and ML libraries
from transformers import (
    TrOCRProcessor, 
    VisionEncoderDecoderModel,
    Trainer, 
    TrainingArguments,
    default_data_collator
)
from datasets import Dataset, DatasetDict
from torch.utils.data import DataLoader
import evaluate
from tqdm import tqdm

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

In [ ]:
# Configuration for the fine-tuning process
class TrOCRConfig:
    # Paths
    BASE_DIR = Path.cwd()
    DATA_DIR = BASE_DIR / "prescription_data"
    IMAGES_DIR = DATA_DIR / "images"
    ANNOTATIONS_FILE = DATA_DIR / "annotations.json"
    OUTPUT_DIR = BASE_DIR / "trocr_prescription_model"
    
    # Model settings
    MODEL_NAME = "microsoft/trocr-base-handwritten"  # Pre-trained on handwritten text
    MAX_TARGET_LENGTH = 256  # Maximum length of prescription text
    
    # Training parameters
    BATCH_SIZE = 4  # Adjust based on GPU memory
    LEARNING_RATE = 5e-5
    NUM_EPOCHS = 10
    WARMUP_STEPS = 100
    EVAL_STEPS = 50
    SAVE_STEPS = 100
    
    # Data split
    TRAIN_SPLIT = 0.8
    VAL_SPLIT = 0.1
    TEST_SPLIT = 0.1

config = TrOCRConfig()

# Create directories if they don't exist
config.DATA_DIR.mkdir(exist_ok=True)
config.IMAGES_DIR.mkdir(exist_ok=True)
config.OUTPUT_DIR.mkdir(exist_ok=True)

print("Configuration setup complete!")
print(f"Data directory: {config.DATA_DIR}")
print(f"Model will be saved to: {config.OUTPUT_DIR}")

In [ ]:
# Data preparation utilities
class PrescriptionDatasetCreator:
    """Helper class to create and manage prescription dataset"""
    
    def __init__(self, images_dir: Path, annotations_file: Path):
        self.images_dir = images_dir
        self.annotations_file = annotations_file
    
    def create_sample_annotations(self):
        """Create a sample annotations file structure for reference"""
        sample_annotations = {
            "dataset_info": {
                "description": "Handwritten prescription OCR dataset",
                "version": "1.0",
                "total_images": 100
            },
            "annotations": [
                {
                    "image_id": "prescription_001.jpg",
                    "text": "Aspirin 100mg twice daily\nMetformin 500mg once daily\nLisinopril 10mg once daily",
                    "drugs_extracted": ["Aspirin", "Metformin", "Lisinopril"],
                    "dosages": ["100mg", "500mg", "10mg"],
                    "frequencies": ["twice daily", "once daily", "once daily"]
                },
                {
                    "image_id": "prescription_002.jpg",
                    "text": "Amoxicillin 250mg three times daily\nIbuprofen 400mg as needed",
                    "drugs_extracted": ["Amoxicillin", "Ibuprofen"],
                    "dosages": ["250mg", "400mg"],
                    "frequencies": ["three times daily", "as needed"]
                }
            ]
        }
        
        with open(self.annotations_file, 'w') as f:
            json.dump(sample_annotations, f, indent=2)
        
        print(f"Sample annotations created at: {self.annotations_file}")
        print("Please replace with your actual annotations!")
    
    def load_annotations(self) -> Dict:
        """Load annotations from JSON file"""
        if not self.annotations_file.exists():
            print("Annotations file not found. Creating sample structure...")
            self.create_sample_annotations()
            return None
        
        with open(self.annotations_file, 'r') as f:
            annotations = json.load(f)
        
        print(f"Loaded {len(annotations['annotations'])} annotations")
        return annotations
    
    def validate_dataset(self) -> bool:
        """Validate that images and annotations match"""
        annotations = self.load_annotations()
        if not annotations:
            return False
        
        # Check if all annotated images exist
        missing_images = []
        for ann in annotations['annotations']:
            image_path = self.images_dir / ann['image_id']
            if not image_path.exists():
                missing_images.append(ann['image_id'])
        
        if missing_images:
            print(f"Warning: Missing images: {missing_images}")
            return False
        
        print("✓ Dataset validation passed")
        return True
    
    def get_dataset_stats(self) -> Dict:
        """Get statistics about the dataset"""
        annotations = self.load_annotations()
        if not annotations:
            return {}
        
        texts = [ann['text'] for ann in annotations['annotations']]
        text_lengths = [len(text) for text in texts]
        
        # Extract all drugs mentioned
        all_drugs = []
        for ann in annotations['annotations']:
            if 'drugs_extracted' in ann:
                all_drugs.extend(ann['drugs_extracted'])
        
        stats = {
            'total_prescriptions': len(annotations['annotations']),
            'avg_text_length': np.mean(text_lengths),
            'max_text_length': max(text_lengths),
            'min_text_length': min(text_lengths),
            'unique_drugs': len(set(all_drugs)),
            'total_drug_mentions': len(all_drugs),
            'most_common_drugs': pd.Series(all_drugs).value_counts().head(10).to_dict()
        }
        
        return stats

# Initialize dataset creator
dataset_creator = PrescriptionDatasetCreator(config.IMAGES_DIR, config.ANNOTATIONS_FILE)
print("Dataset creator initialized!")

In [ ]:
# Check and prepare your dataset
print("=== Dataset Preparation ===")

# Check if images directory has files
image_files = list(config.IMAGES_DIR.glob('*.jpg')) + list(config.IMAGES_DIR.glob('*.png')) + list(config.IMAGES_DIR.glob('*.jpeg'))
print(f"Found {len(image_files)} image files in {config.IMAGES_DIR}")

if len(image_files) == 0:
    print("\n⚠️  No images found! Please:")
    print(f"1. Add your prescription images to: {config.IMAGES_DIR}")
    print("2. Supported formats: .jpg, .png, .jpeg")
    print("3. Run this cell again after adding images")
else:
    print(f"✓ Images found: {[f.name for f in image_files[:5]]}...")
    
    # Validate or create annotations
    if dataset_creator.validate_dataset():
        stats = dataset_creator.get_dataset_stats()
        print("\n=== Dataset Statistics ===")
        for key, value in stats.items():
            print(f"{key}: {value}")
    else:
        print("\n⚠️  Dataset validation failed or annotations not found!")
        print("Creating sample annotations structure...")
        dataset_creator.create_sample_annotations()
        print("\nPlease:")
        print(f"1. Edit the annotations file: {config.ANNOTATIONS_FILE}")
        print("2. Add transcriptions for all your prescription images")
        print("3. Run this cell again after creating proper annotations")

In [ ]:
# Load and initialize the TrOCR model
print("=== Loading TrOCR Model ===")

# Load processor and model
processor = TrOCRProcessor.from_pretrained(config.MODEL_NAME)
model = VisionEncoderDecoderModel.from_pretrained(config.MODEL_NAME)

# Move model to device
model.to(device)

print(f"✓ Loaded model: {config.MODEL_NAME}")
print(f"✓ Model parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"✓ Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

# Model configuration
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id
model.config.vocab_size = model.config.decoder.vocab_size

# Set generation parameters for better prescription text generation
model.config.eos_token_id = processor.tokenizer.sep_token_id
model.config.max_length = config.MAX_TARGET_LENGTH
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0
model.config.num_beams = 4

print("✓ Model configuration updated for prescription text generation")

In [ ]:
# Custom Dataset class for prescription images
class PrescriptionOCRDataset:
    """Custom dataset class for prescription OCR"""
    
    def __init__(self, annotations: List[Dict], images_dir: Path, processor, max_target_length: int = 256):
        self.annotations = annotations
        self.images_dir = images_dir
        self.processor = processor
        self.max_target_length = max_target_length
    
    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self, idx):
        annotation = self.annotations[idx]
        
        # Load image
        image_path = self.images_dir / annotation['image_id']
        image = Image.open(image_path).convert('RGB')
        
        # Process image
        pixel_values = self.processor(image, return_tensors="pt").pixel_values.squeeze()
        
        # Process text
        text = annotation['text']
        labels = self.processor.tokenizer(
            text, 
            padding="max_length", 
            max_length=self.max_target_length, 
            truncation=True, 
            return_tensors="pt"
        ).input_ids.squeeze()
        
        # Replace padding token id with -100 for loss computation
        labels[labels == self.processor.tokenizer.pad_token_id] = -100
        
        return {
            'pixel_values': pixel_values,
            'labels': labels
        }

def create_datasets(annotations_data: Dict, images_dir: Path, processor, config: TrOCRConfig):
    """Create train/val/test datasets"""
    annotations = annotations_data['annotations']
    
    # Split the data
    np.random.shuffle(annotations)
    
    n_total = len(annotations)
    n_train = int(n_total * config.TRAIN_SPLIT)
    n_val = int(n_total * config.VAL_SPLIT)
    
    train_annotations = annotations[:n_train]
    val_annotations = annotations[n_train:n_train + n_val]
    test_annotations = annotations[n_train + n_val:]
    
    print(f"Dataset split: Train={len(train_annotations)}, Val={len(val_annotations)}, Test={len(test_annotations)}")
    
    # Create datasets
    train_dataset = PrescriptionOCRDataset(train_annotations, images_dir, processor, config.MAX_TARGET_LENGTH)
    val_dataset = PrescriptionOCRDataset(val_annotations, images_dir, processor, config.MAX_TARGET_LENGTH)
    test_dataset = PrescriptionOCRDataset(test_annotations, images_dir, processor, config.MAX_TARGET_LENGTH)
    
    return train_dataset, val_dataset, test_dataset

print("Custom dataset classes defined!")

In [ ]:
# Prepare datasets for training (only if annotations are available)
annotations_data = dataset_creator.load_annotations()

if annotations_data and len(annotations_data['annotations']) > 0:
    print("=== Preparing Training Datasets ===")
    
    # Create datasets
    train_dataset, val_dataset, test_dataset = create_datasets(
        annotations_data, config.IMAGES_DIR, processor, config
    )
    
    print(f"✓ Training dataset: {len(train_dataset)} samples")
    print(f"✓ Validation dataset: {len(val_dataset)} samples")
    print(f"✓ Test dataset: {len(test_dataset)} samples")
    
    # Test data loading
    if len(train_dataset) > 0:
        sample = train_dataset[0]
        print(f"\nSample data shapes:")
        print(f"Image (pixel_values): {sample['pixel_values'].shape}")
        print(f"Labels: {sample['labels'].shape}")
        
        # Decode sample text
        sample_text = processor.tokenizer.decode(sample['labels'], skip_special_tokens=True)
        print(f"Sample text: '{sample_text[:100]}...'")
    
    datasets_ready = True
else:
    print("❌ No valid annotations found. Cannot proceed with training.")
    print("Please ensure you have:")
    print("1. Added prescription images to the images directory")
    print("2. Created proper annotations with transcribed text")
    datasets_ready = False

In [ ]:
# Define evaluation metrics
def compute_metrics(eval_pred):
    """Compute evaluation metrics for OCR task"""
    predictions, labels = eval_pred
    
    # Decode predictions and labels
    decoded_preds = processor.tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
    # Replace -100 in labels as they are used for padding
    labels = np.where(labels != -100, labels, processor.tokenizer.pad_token_id)
    decoded_labels = processor.tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Compute character-level accuracy
    char_accuracy = []
    word_accuracy = []
    
    for pred, label in zip(decoded_preds, decoded_labels):
        # Character-level accuracy
        if len(label) > 0:
            char_acc = sum(c1 == c2 for c1, c2 in zip(pred, label)) / max(len(pred), len(label))
            char_accuracy.append(char_acc)
        
        # Word-level accuracy
        pred_words = set(pred.lower().split())
        label_words = set(label.lower().split())
        if len(label_words) > 0:
            word_acc = len(pred_words & label_words) / len(label_words)
            word_accuracy.append(word_acc)
    
    return {
        'char_accuracy': np.mean(char_accuracy) if char_accuracy else 0.0,
        'word_accuracy': np.mean(word_accuracy) if word_accuracy else 0.0,
    }

# Custom trainer for better logging
class PrescriptionOCRTrainer(Trainer):
    def log(self, logs):
        if hasattr(self, '_log_history'):
            self._log_history.append(logs)
        else:
            self._log_history = [logs]
        super().log(logs)

print("Evaluation metrics and custom trainer defined!")

In [ ]:
# Training setup and execution
if datasets_ready:
    print("=== Setting Up Training ===")
    
    # Training arguments
    training_args = TrainingArguments(
        output_dir=str(config.OUTPUT_DIR),
        per_device_train_batch_size=config.BATCH_SIZE,
        per_device_eval_batch_size=config.BATCH_SIZE,
        num_train_epochs=config.NUM_EPOCHS,
        learning_rate=config.LEARNING_RATE,
        warmup_steps=config.WARMUP_STEPS,
        logging_steps=10,
        eval_steps=config.EVAL_STEPS,
        evaluation_strategy="steps",
        save_steps=config.SAVE_STEPS,
        save_strategy="steps",
        load_best_model_at_end=True,
        metric_for_best_model="char_accuracy",
        greater_is_better=True,
        remove_unused_columns=False,
        dataloader_pin_memory=False,
        fp16=torch.cuda.is_available(),  # Use mixed precision if GPU available
        report_to="none",  # Disable wandb/tensorboard for simplicity
        seed=42,
    )
    
    # Initialize trainer
    trainer = PrescriptionOCRTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics,
        data_collator=default_data_collator,
    )
    
    print(f"✓ Trainer initialized with {len(train_dataset)} training samples")
    print(f"✓ GPU memory usage: {torch.cuda.memory_allocated() / 1024**3:.1f} GB" if torch.cuda.is_available() else "✓ Using CPU")
    
    # Start training
    print("\n🚀 Starting training...")
    print(f"Training for {config.NUM_EPOCHS} epochs with batch size {config.BATCH_SIZE}")
    
    try:
        trainer.train()
        print("\n✅ Training completed successfully!")
        
        # Save the final model
        trainer.save_model()
        processor.save_pretrained(config.OUTPUT_DIR)
        
        print(f"✓ Model saved to: {config.OUTPUT_DIR}")
        
    except Exception as e:
        print(f"❌ Training failed: {str(e)}")
        print("This might be due to:")
        print("- Insufficient GPU memory (try reducing batch size)")
        print("- Corrupted images in the dataset")
        print("- Invalid annotations format")
        
else:
    print("❌ Cannot start training - datasets not ready")
    print("Please complete the dataset preparation steps first")

In [ ]:
# Model evaluation and testing
if datasets_ready and len(test_dataset) > 0:
    print("=== Evaluating Fine-tuned Model ===")
    
    # Load the best model if training was successful
    try:
        model_path = config.OUTPUT_DIR
        if (model_path / "pytorch_model.bin").exists():
            print("Loading fine-tuned model...")
            fine_tuned_model = VisionEncoderDecoderModel.from_pretrained(model_path)
            fine_tuned_processor = TrOCRProcessor.from_pretrained(model_path)
            fine_tuned_model.to(device)
        else:
            print("Using current model (training may not have completed)")
            fine_tuned_model = model
            fine_tuned_processor = processor
        
        # Evaluate on test set
        fine_tuned_model.eval()
        test_predictions = []
        test_ground_truth = []
        
        print(f"Testing on {len(test_dataset)} samples...")
        
        with torch.no_grad():
            for i in tqdm(range(len(test_dataset)), desc="Evaluating"):
                sample = test_dataset[i]
                
                # Get prediction
                pixel_values = sample['pixel_values'].unsqueeze(0).to(device)
                generated_ids = fine_tuned_model.generate(pixel_values, max_length=config.MAX_TARGET_LENGTH)
                generated_text = fine_tuned_processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
                
                # Get ground truth
                labels = sample['labels']
                labels = labels[labels != -100]  # Remove padding
                ground_truth = fine_tuned_processor.tokenizer.decode(labels, skip_special_tokens=True)
                
                test_predictions.append(generated_text)
                test_ground_truth.append(ground_truth)
        
        # Calculate overall metrics
        char_accuracies = []
        word_accuracies = []
        exact_matches = 0
        
        for pred, truth in zip(test_predictions, test_ground_truth):
            # Character accuracy
            if len(truth) > 0:
                char_acc = sum(c1 == c2 for c1, c2 in zip(pred, truth)) / max(len(pred), len(truth))
                char_accuracies.append(char_acc)
            
            # Word accuracy
            pred_words = set(pred.lower().split())
            truth_words = set(truth.lower().split())
            if len(truth_words) > 0:
                word_acc = len(pred_words & truth_words) / len(truth_words)
                word_accuracies.append(word_acc)
            
            # Exact match
            if pred.strip().lower() == truth.strip().lower():
                exact_matches += 1
        
        # Print results
        print("\n=== Test Results ===")
        print(f"Character Accuracy: {np.mean(char_accuracies):.3f} ± {np.std(char_accuracies):.3f}")
        print(f"Word Accuracy: {np.mean(word_accuracies):.3f} ± {np.std(word_accuracies):.3f}")
        print(f"Exact Match Accuracy: {exact_matches / len(test_predictions):.3f} ({exact_matches}/{len(test_predictions)})")
        
        # Show sample predictions
        print("\n=== Sample Predictions ===")
        for i in range(min(3, len(test_predictions))):
            print(f"\nSample {i+1}:")
            print(f"Ground Truth: {test_ground_truth[i][:100]}...")
            print(f"Prediction:   {test_predictions[i][:100]}...")
            
    except Exception as e:
        print(f"❌ Evaluation failed: {str(e)}")
else:
    print("❌ Cannot evaluate - no test dataset available")

In [ ]:
# Drug extraction utility for Zaura Health integration
class PrescriptionDrugExtractor:
    """Extract structured drug information from OCR text for Zaura Health"""
    
    def __init__(self):
        # Common drug name patterns
        self.drug_patterns = [
            r'([A-Za-z][A-Za-z0-9]+)\s+(\d+(?:\.\d+)?\s*(?:mg|g|ml|mcg|units?))',
            r'([A-Za-z][A-Za-z0-9]+)\s+([0-9\.]+)\s*(mg|g|ml|mcg|units?)',
        ]
        
        # Frequency patterns
        self.frequency_patterns = [
            r'(once|twice|three times|four times)\s+(daily|per day)',
            r'(\d+)\s*times?\s+(daily|per day|a day)',
            r'every\s+(\d+)\s*hours?',
            r'as needed',
            r'PRN'
        ]
    
    def extract_medications(self, ocr_text: str) -> Dict:
        """Extract structured medication information from OCR text"""
        medications = []
        
        # Split text into lines
        lines = ocr_text.strip().split('\n')
        
        for line in lines:
            line = line.strip()
            if not line:
                continue
            
            # Extract drug name and dosage
            drug_match = None
            for pattern in self.drug_patterns:
                match = re.search(pattern, line, re.IGNORECASE)
                if match:
                    drug_match = match
                    break
            
            if drug_match:
                drug_name = drug_match.group(1)
                dosage = drug_match.group(2)
                
                # Extract frequency
                frequency = "as needed"  # default
                for freq_pattern in self.frequency_patterns:
                    freq_match = re.search(freq_pattern, line, re.IGNORECASE)
                    if freq_match:
                        frequency = freq_match.group(0)
                        break
                
                medications.append({
                    'drug_name': drug_name,
                    'dosage': dosage,
                    'frequency': frequency,
                    'raw_text': line
                })
        
        return {
            'medications': medications,
            'total_medications': len(medications),
            'raw_ocr_text': ocr_text
        }
    
    def format_for_zaura_health(self, extracted_data: Dict) -> Dict:
        """Format extracted data for Zaura Health drug interaction model"""
        drugs = [med['drug_name'] for med in extracted_data['medications']]
        
        # Calculate total daily doses (simplified)
        total_doses = 0
        for med in extracted_data['medications']:
            freq_text = med['frequency'].lower()
            if 'once' in freq_text:
                total_doses += 1
            elif 'twice' in freq_text:
                total_doses += 2
            elif 'three' in freq_text:
                total_doses += 3
            else:
                total_doses += 1  # default
        
        zaura_format = {
            'drugs': drugs[:10],  # Limit to 10 drugs (model constraint)
            'total_drugs': len(drugs),
            'doses_per_24_hrs': total_doses,
            'has_dosage_info': 1 if any('mg' in med['dosage'] for med in extracted_data['medications']) else 0,
            'extracted_medications': extracted_data['medications']
        }
        
        return zaura_format

# Initialize drug extractor
drug_extractor = PrescriptionDrugExtractor()
print("Drug extraction utility ready for Zaura Health integration!")

In [ ]:
# Complete pipeline: Image → OCR → Drug Extraction → Zaura Health Format
def process_prescription_image(image_path: str, model, processor) -> Dict:
    """Complete pipeline to process a prescription image"""
    try:
        # Load and preprocess image
        image = Image.open(image_path).convert('RGB')
        
        # OCR with fine-tuned model
        pixel_values = processor(image, return_tensors="pt").pixel_values.to(device)
        
        model.eval()
        with torch.no_grad():
            generated_ids = model.generate(
                pixel_values,
                max_length=config.MAX_TARGET_LENGTH,
                num_beams=4,
                early_stopping=True
            )
        
        ocr_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
        
        # Extract drug information
        extracted_data = drug_extractor.extract_medications(ocr_text)
        
        # Format for Zaura Health
        zaura_format = drug_extractor.format_for_zaura_health(extracted_data)
        
        return {
            'success': True,
            'ocr_text': ocr_text,
            'extracted_medications': extracted_data,
            'zaura_health_format': zaura_format
        }
        
    except Exception as e:
        return {
            'success': False,
            'error': str(e)
        }

# Test the complete pipeline if we have a trained model
if datasets_ready and (config.OUTPUT_DIR / "pytorch_model.bin").exists():
    print("=== Testing Complete Pipeline ===")
    
    # Load fine-tuned model
    test_model = VisionEncoderDecoderModel.from_pretrained(config.OUTPUT_DIR)
    test_processor = TrOCRProcessor.from_pretrained(config.OUTPUT_DIR)
    test_model.to(device)
    
    # Test with a sample image
    sample_images = list(config.IMAGES_DIR.glob('*.jpg'))[:3]  # Test first 3 images
    
    for i, img_path in enumerate(sample_images):
        print(f"\n--- Processing {img_path.name} ---")
        result = process_prescription_image(str(img_path), test_model, test_processor)
        
        if result['success']:
            print(f"OCR Text: {result['ocr_text'][:100]}...")
            print(f"Extracted Drugs: {[med['drug_name'] for med in result['extracted_medications']['medications']]}")
            print(f"Zaura Health Format: {result['zaura_health_format']['drugs']}")
        else:
            print(f"Error: {result['error']}")

else:
    print("Complete pipeline test skipped - model not trained yet")
    print("The pipeline function is ready for use after training completion")

In [ ]:
# Create API endpoint for Zaura Health website integration
api_code = '''
# Save this as prescription_ocr_api.py
from flask import Flask, request, jsonify
from PIL import Image
import torch
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
import io
import base64
import re
from typing import Dict

app = Flask(__name__)

# Load model once when server starts
MODEL_PATH = "./trocr_prescription_model"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

try:
    processor = TrOCRProcessor.from_pretrained(MODEL_PATH)
    model = VisionEncoderDecoderModel.from_pretrained(MODEL_PATH)
    model.to(device)
    model.eval()
    print(f"Model loaded successfully on {device}")
except:
    # Fallback to base model if fine-tuned model not available
    processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")
    model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-handwritten")
    model.to(device)
    model.eval()
    print("Using base TrOCR model (fine-tuned model not found)")

class DrugExtractor:
    def __init__(self):
        self.drug_patterns = [
            r'([A-Za-z][A-Za-z0-9]+)\\s+(\\d+(?:\\.\\d+)?\\s*(?:mg|g|ml|mcg|units?))',
            r'([A-Za-z][A-Za-z0-9]+)\\s+([0-9\\.]+)\\s*(mg|g|ml|mcg|units?)'
        ]
    
    def extract_medications(self, ocr_text: str) -> Dict:
        medications = []
        lines = ocr_text.strip().split('\\n')
        
        for line in lines:
            line = line.strip()
            if not line:
                continue
            
            for pattern in self.drug_patterns:
                match = re.search(pattern, line, re.IGNORECASE)
                if match:
                    medications.append({
                        'drug_name': match.group(1),
                        'dosage': match.group(2) if len(match.groups()) > 1 else '',
                        'raw_text': line
                    })
                    break
        
        return medications

drug_extractor = DrugExtractor()

@app.route('/ocr', methods=['POST'])
def process_prescription():
    try:
        # Get image from request
        if 'image' not in request.files:
            return jsonify({'error': 'No image provided'}), 400
        
        image_file = request.files['image']
        image = Image.open(image_file.stream).convert('RGB')
        
        # Process with TrOCR
        pixel_values = processor(image, return_tensors="pt").pixel_values.to(device)
        
        with torch.no_grad():
            generated_ids = model.generate(pixel_values, max_length=256)
        
        ocr_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
        
        # Extract medications
        medications = drug_extractor.extract_medications(ocr_text)
        drug_names = [med['drug_name'] for med in medications]
        
        return jsonify({
            'success': True,
            'ocr_text': ocr_text,
            'drugs': drug_names[:10],  # Limit to 10 for Zaura Health model
            'total_drugs': len(drug_names),
            'medications': medications
        })
        
    except Exception as e:
        return jsonify({
            'success': False,
            'error': str(e)
        }), 500

@app.route('/health', methods=['GET'])
def health_check():
    return jsonify({'status': 'healthy', 'model_loaded': True})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, debug=True)
'''

# Save API code to file
with open('prescription_ocr_api.py', 'w') as f:
    f.write(api_code)

print("\n=== API Integration Code Generated ===")
print("✓ Flask API saved as: prescription_ocr_api.py")
print("\nTo use with Zaura Health:")
print("1. Run: python prescription_ocr_api.py")
print("2. API will be available at: http://localhost:5000")
print("3. POST prescription images to: /ocr")
print("4. Health check at: /health")

# Integration example for Zaura Health frontend
integration_js = '''
// JavaScript for Zaura Health website integration
async function uploadPrescription(imageFile) {
    const formData = new FormData();
    formData.append('image', imageFile);
    
    try {
        const response = await fetch('http://localhost:5000/ocr', {
            method: 'POST',
            body: formData
        });
        
        const result = await response.json();
        
        if (result.success) {
            // Populate drug input fields with extracted data
            populateDrugFields(result.drugs);
            showOCRResult(result.ocr_text);
        } else {
            showError(result.error);
        }
    } catch (error) {
        showError('Failed to process prescription: ' + error.message);
    }
}

function populateDrugFields(drugs) {
    // Fill the drug input fields in Zaura Health form
    for (let i = 0; i < Math.min(drugs.length, 10); i++) {
        const drugInput = document.getElementById(`drug${i+1}`);
        if (drugInput) {
            drugInput.value = drugs[i];
        }
    }
}
'''

with open('zaura_health_integration.js', 'w') as f:
    f.write(integration_js)

print("\n✓ JavaScript integration saved as: zaura_health_integration.js")
print("\n=== Complete Integration Ready! ===")

In [ ]:
# Summary and next steps
print("\n" + "="*80)
print("TROCR PRESCRIPTION OCR - SETUP COMPLETE")
print("="*80)

print("\n📁 DIRECTORY STRUCTURE:")
print(f"├── {config.DATA_DIR}/")
print(f"│   ├── images/          # Your prescription images")
print(f"│   └── annotations.json # Ground truth transcriptions")
print(f"├── {config.OUTPUT_DIR}/ # Fine-tuned model")
print("├── prescription_ocr_api.py      # Flask API")
└── zaura_health_integration.js  # Frontend integration")

print("\n🚀 NEXT STEPS:")
print("\n1. PREPARE YOUR DATA:")
print(f"   • Add prescription images to: {config.IMAGES_DIR}")
print(f"   • Create transcriptions in: {config.ANNOTATIONS_FILE}")

print("\n2. FINE-TUNE THE MODEL:")
print("   • Run all cells in this notebook")
print("   • Monitor training progress and metrics")
print("   • Adjust hyperparameters if needed")

print("\n3. DEPLOY TO ZAURA HEALTH:")
print("   • Run: python prescription_ocr_api.py")
print("   • Integrate with Zaura Health frontend")
print("   • Test end-to-end pipeline")

print("\n📊 EXPECTED PERFORMANCE:")
print("• Character accuracy: 85-95% (depending on image quality)")
print("• Word accuracy: 80-90% (for medical terms)")
print("• Processing time: ~1-3 seconds per image")

print("\n⚡ OPTIMIZATION TIPS:")
print("• Use high-quality, well-lit prescription images")
print("• Ensure handwriting is legible in training data")
print("• Add domain-specific medical terms to training")
print("• Use GPU for faster training and inference")

print("\n✅ Ready to process handwritten prescriptions for Zaura Health!")